# ARIMA untuk Forecast Ayam Tua (14 hari ke depan)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
from pathlib import Path

## Load Data

In [ ]:
possible_paths = [
    Path("processed_for_model/ts_ayam_tua_clean.csv"),
    Path("notebooks/processed_for_model/ts_ayam_tua_clean.csv"),
]

file_path = None
for p in possible_paths:
    if p.exists():
        file_path = str(p)
        break

if file_path is None:
    raise FileNotFoundError("Tidak menemukan ts_ayam_tua_clean.csv")

df = pd.read_csv(file_path, parse_dates=['date'])
df = df.set_index('date').sort_index()

print("Data loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst 5 rows:")
print(df.head())

## Visualisasi Data

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(df['Ayam_Tua'])
plt.title('Time Series Ayam Tua')
plt.xlabel('Tanggal')
plt.ylabel('Jumlah (kg)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Build Model ARIMA(2,1,2)

In [ ]:
series = df['Ayam_Tua']
model = ARIMA(series, order=(2, 1, 2))
model_fit = model.fit()

print("Model ARIMA(2,1,2) berhasil di-fit!")
print("\nRingkasan Model:")
print(model_fit.summary())

## Forecast 14 Hari

In [ ]:
n_forecast = 14
forecast_values = model_fit.forecast(steps=n_forecast)

start_date = datetime(2025, 1, 2)

prediction_dates = []
current_date = start_date
days_added = 0

while days_added < n_forecast:
    if current_date.month == 1 and current_date.day == 1:
        current_date += timedelta(days=1)
        continue
    
    prediction_dates.append(current_date)
    days_added += 1
    current_date += timedelta(days=1)

forecast_series = pd.Series(
    forecast_values.values,
    index=prediction_dates,
    name='Forecast_Ayam_Tua'
)

print(f"\nForecast untuk {len(forecast_series)} hari:")
print(f"Periode: {forecast_series.index[0].strftime('%Y-%m-%d')} - {forecast_series.index[-1].strftime('%Y-%m-%d')}")
print(f"\nRata-rata prediksi: {forecast_series.mean():.2f} kg/hari")
print(f"\nDetail prediksi:")
for date, value in forecast_series.items():
    print(f"{date.strftime('%Y-%m-%d')}: {value:.2f} kg")

## Visualisasi Historis + Forecast

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(df['Ayam_Tua'].tail(30), label='Historis (30 hari terakhir)', marker='o', markersize=3)
plt.plot(forecast_series, label='Forecast 14 Hari', 
         linestyle='--', marker='s', markersize=4, color='red')
plt.title('Historis vs Forecast Ayam Tua\n')
plt.xlabel('Tanggal')
plt.ylabel('Jumlah (kg)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Simpan Hasil Forecast

In [ ]:
output_forecast = forecast_series.reset_index()
output_forecast.columns = ['date', 'Forecast_Ayam_Tua']
output_path = "forecast_ayam_tua_14hari_updated.csv"
output_forecast.to_csv(output_path, index=False)

print(f"\nForecast berhasil disimpan ke: {output_path}")
print(f"\nCatatan: Periode 2-15 Januari 2025")